<a href="https://colab.research.google.com/github/ovimasbul83/ovimasbul/blob/main/my_unet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d vikramtiwari/pix2pix-dataset

100% 2.39G/2.40G [00:10<00:00, 235MB/s]
100% 2.40G/2.40G [00:10<00:00, 245MB/s]


In [ ]:
!unzip /content/pix2pix-dataset.zip -d /content/pix2pix-dataset

Streaming output truncated to the last 5000 lines.
  inflating: /content/pix2pix-dataset/edges2shoes/edges2shoes/train/8200_AB.jpg  
  inflating: /content/pix2pix-dataset/edges2shoes/edges2shoes/train/8201_AB.jpg  
  inflating: /content/pix2pix-dataset/edges2shoes/edges2shoes/train/8202_AB.jpg  
  inflating: /content/pix2pix-dataset/edges2shoes/edges2shoes/train/8203_AB.jpg  
  inflating: /content/pix2pix-dataset/edges2shoes/edges2shoes/train/8204_AB.jpg  
  inflating: /content/pix2pix-dataset/edges2shoes/edges2shoes/train/8205_AB.jpg  
  inflating: /content/pix2pix-dataset/edges2shoes/edges2shoes/train/8206_AB.jpg  
  inflating: /content/pix2pix-dataset/edges2shoes/edges2shoes/train/8207_AB.jpg  
  inflating: /content/pix2pix-dataset/edges2shoes/edges2shoes/train/8208_AB.jpg  
  inflating: /content/pix2pix-dataset/edges2shoes/edges2shoes/train/8209_AB.jpg  
  inflating: /content/pix2pix-dataset/edges2shoes/edges2shoes/train/820_AB.jpg  
  inflating: /content/pix2pix-dataset/edges2shoe

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import cv2
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torch
from torch.nn import functional as F
from torchvision import datasets,transforms
import os
import tqdm
import torch.nn as nn
from torchvision.utils import make_grid

In [ ]:

'''for img,label in dataloader:
  print( img.shape,label.shape)
  print(img[0].min(), img[0].max())
  print(label[0].min(), label[0].max())
  fig, ax = plt.subplots(1, 2, figsize=(15, 6))
  ax[0].imshow(img[0].permute(1, 2, 0))
  ax[1].imshow(label[0].permute(1,2,0))
  break'''
  

'for img,label in dataloader:\n  print( img.shape,label.shape)\n  print(img[0].min(), img[0].max())\n  print(label[0].min(), label[0].max())\n  fig, ax = plt.subplots(1, 2, figsize=(15, 6))\n  ax[0].imshow(img[0].permute(1, 2, 0))\n  ax[1].imshow(label[0].permute(1,2,0))\n  break'

In [ ]:
class downsamplingblock(nn.Module):
  def __init__(self,input_channels,drpout=False,bn=True):
    super(downsamplingblock,self).__init__()
    self.conv_1=nn.Conv2d(input_channels,input_channels*2,kernel_size=3,padding=1)
    self.conv_2=nn.Conv2d(input_channels*2,input_channels*2,kernel_size=3,padding=1)
    self.activation=nn.LeakyReLU(0.2)
    self.maxpool=nn.MaxPool2d(kernel_size=2,stride=2)
    if bn:
      self.batchnorm=nn.BatchNorm2d(input_channels*2)
    self.bn=bn;
    if drpout:
      self.dropout=nn.Dropout()
    self.drpout=drpout
  def forward(self,x):
    x=self.conv_1(x)
    if self.bn:
      x=self.batchnorm(x)
    if  self.drpout:
      x=self.dropout(x)
    x=self.activation(x)
    x=self.conv_2(x)
    if self.bn:
      x=self.batchnorm(x)
    if  self.drpout:
      x=self.dropout(x)
    x=self.maxpool(x)
    return x
def crop(image, new_shape):
    middle_height = image.shape[2] // 2
    starting_height = middle_height - round(new_shape[2] / 2)
    final_height = starting_height + new_shape[2]
    cropped_image = image[:, :, starting_height:final_height, starting_height:final_height]
    return cropped_image


class featuremapblock(nn.Module):
  def __init__(self,input_channels,output_channels):
     super(featuremapblock,self).__init__()
     self.conv=nn.Conv2d(input_channels,output_channels,kernel_size=1)
  def forward(self,x):
    x=self.conv(x)
    return x;
class upsamplingblock(nn.Module):
   
    def __init__(self, input_channels, use_dropout=False, use_bn=True):
        super(upsamplingblock, self).__init__()
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.conv1 = nn.Conv2d(input_channels, input_channels // 2, kernel_size=2)
        self.conv2 = nn.Conv2d(input_channels, input_channels // 2, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(input_channels // 2, input_channels // 2, kernel_size=2, padding=1)
        if use_bn:
            self.batchnorm = nn.BatchNorm2d(input_channels // 2)
        self.use_bn = use_bn
        self.activation = nn.ReLU()
        if use_dropout:
            self.dropout = nn.Dropout()
        self.use_dropout = use_dropout

    def forward(self, x, crp):
        
        x = self.upsample(x)
        x = self.conv1(x)
        crp= crop( crp,x.shape)
        x = torch.cat([x,crp],axis=1)
        x = self.conv2(x)
        if self.use_bn:
            x = self.batchnorm(x)
        if self.use_dropout:
            x = self.dropout(x)
        x = self.activation(x)
        x = self.conv3(x)
        if self.use_bn:
            x = self.batchnorm(x)
        if self.use_dropout:
            x = self.dropout(x)
        x = self.activation(x)
        return x
class unet(nn.Module):
  def __init__(self,input_channels,output_channels,hidden_channels=32):
    super(unet,self).__init__()
    self.upfeature=featuremapblock(input_channels,hidden_channels)
    self.down1=downsamplingblock(hidden_channels,drpout=True)
    self.down2=downsamplingblock(hidden_channels*2)
    self.down3=downsamplingblock(hidden_channels*4,drpout=True)
    self.down4=downsamplingblock(hidden_channels*8)
    self.down5=downsamplingblock(hidden_channels*16,drpout=True)
    self.down6=downsamplingblock(hidden_channels*32)
    self.up1=upsamplingblock(hidden_channels*64)
    self.up2=upsamplingblock(hidden_channels*32)
    self.up3=upsamplingblock(hidden_channels*16)
    self.up4=upsamplingblock(hidden_channels*8)
    self.up5=upsamplingblock(hidden_channels*4)
    self.up6=upsamplingblock(hidden_channels*2)
    self.downfeature=featuremapblock(hidden_channels,output_channels)
    self.sigmoid=torch.nn.Sigmoid()
  def forward(self,x):
      x0=self.upfeature(x)
      #print(x0.shape)
      x1=self.down1(x0)
      #print(x1.shape)
      x2=self.down2(x1)
      #print(x2.shape)
      x3=self.down3(x2)
      #print(x3.shape)
      x4=self.down4(x3)
      #print(x4.shape)
      x5=self.down5(x4)
      #print(x5.shape)
      x6=self.down6(x5)
      #print(x6.shape)
      x7=self.up1(x6,x5)
      #print(x7.shape)
      x8=self.up2(x7,x4)
      #print(x8.shape)
      x9=self.up3(x8,x3)
      #print(x9.shape)
      x10=self.up4(x9,x2)
      #print(x10.shape)
      x11=self.up5(x10,x1)
      #print(x11.shape)
      x12=self.up6(x11,x0)
      #print(x12.shape)
      xl=self.downfeature(x12)
      #print(xl.shape)
      return self.sigmoid(xl)

class patchgan(nn.Module):
  def __init__(self,input_channels,hidden_channels=8):
    super(patchgan,self).__init__()
    self.upfeature=featuremapblock(input_channels,hidden_channels)
    self.down1=downsamplingblock(hidden_channels)
    self.down2=downsamplingblock(hidden_channels*2)
    self.down3=downsamplingblock(hidden_channels*4)
    self.down4=downsamplingblock(hidden_channels*8)
    self.conv=nn.Conv2d(hidden_channels*16,1,kernel_size=1)
  def forward(self,x,y):
    x = torch.cat([x, y], axis=1)
    x0=self.upfeature(x)
    x1=self.down1(x0)
    x2=self.down2(x1)
    x3=self.down3(x2)
    x4=self.down4(x3)
    xn=self.conv(x4)
    return xn




In [ ]:
adv_criterion =nn.BCEWithLogitsLoss()
recon_criterion = nn.SmoothL1Loss()
lambda_pix=200
epochs=30
input_dim=3
real_dim=3
display_step=20
lr=0.0001
target_shape=256
device='cuda'

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
])

import torchvision
from torch.utils.data import Dataset, DataLoader
import os
#dataset = torchvision.datasets.ImageFolder("/content/pix2pix-dataset/facades/facades/train", transform=transform)
data_transforms=transforms.Compose([
                                    transforms.ToTensor(),
                                    transforms.Resize((256,256)),
                                    ])
root='/content/pix2pix-dataset/facades/facades/train'
rootcity='/content/pix2pix-dataset/cityscapes/cityscapes/train'
rootedge='/content/pix2pix-dataset/edges2shoes/edges2shoes/train'
rootmap='/content/pix2pix-dataset/maps/maps/train'
class fascadeDataSet(Dataset):
    def __init__(self, path):
        self.path = path
        self.list_files = os.listdir(path)
            
    def __len__(self):
        return len(self.list_files)
     
    def __getitem__(self, index):
        img_file = self.list_files[index]
        img_path = os.path.join(self.path, img_file)
        image = np.array(Image.open(img_path))
        h,w,c=image.shape
        input_image = image[:, :int(w/2), :]
        target_image = image[:, int(w/2):, :]
        input_image=data_transforms(input_image)
        target_image=data_transforms(target_image)
        return target_image, input_image
data=fascadeDataSet(root)

In [ ]:

gen=unet(input_dim,real_dim).to(device)
gen_opt=torch.optim.Adam(gen.parameters(),lr=lr)
disc=patchgan(input_dim+real_dim).to(device)
disc_opt=torch.optim.Adam(disc.parameters(),lr=lr)


In [ ]:

def get_gen_loss(gen,disc,real,condition,adv_criterion,recon_criterion,lambda_pix):
    fake_image = gen(condition)
    disc_fake = disc(fake_image, condition)
    gen_adv_loss = adv_criterion(disc_fake, torch.ones_like(disc_fake))
    gen_recon_loss = recon_criterion(real, fake_image)
    gen_loss = gen_adv_loss + lambda_pix* gen_recon_loss
    #### END CODE HERE ####
    return gen_loss

In [ ]:
'''def show_tensor_images(image_tensor, num_images=25, size=(1, 28, 28)):
    image_shifted = image_tensor
    image_unflat = image_shifted.detach().cpu().view(-1, *size)
    image_grid = make_grid(image_unflat[:num_images], nrow=5)
    plt.imshow(image_grid.permute(1, 2, 0).squeeze())
    plt.show()'''

def show_tensor_images(image_tensor, num_images=25,xlabel=""):

    plt.figure(figsize=(12,12))
    #image_tensor = ((image_tensor + 1.0) / 2.0)
    image_unflat = image_tensor.detach().cpu()
    image_grid = make_grid(image_unflat[:num_images], nrow=5)
    plt.imshow(image_grid.permute(1,2,0).squeeze())
    plt.xticks([])
    plt.yticks([])
    plt.xlabel(xlabel)
    plt.show()

In [ ]:


mean_generator_loss = 0
mean_discriminator_loss = 0
dataloader = DataLoader(data, batch_size=4, shuffle=True)
cur_step = 0

for epoch in range(epochs):

        for condition, real in dataloader:
     
            condition=F.interpolate(condition,size=target_shape)
            real=F.interpolate(real,size=target_shape)
            cur_batch_size=len(condition)
            condition = condition.to(device)
            real = real.to(device)
            disc_opt.zero_grad()

        
            disc_opt.zero_grad() 
            with torch.no_grad():
                fake = gen(condition)
            disc_fake_hat = disc(fake.detach(), condition) 
            disc_fake_loss = adv_criterion(disc_fake_hat, torch.zeros_like(disc_fake_hat))
            disc_real_hat = disc(real, condition)
            disc_real_loss = adv_criterion(disc_real_hat, torch.ones_like(disc_real_hat))
            disc_loss = (disc_fake_loss + disc_real_loss) / 2
            disc_loss.backward(retain_graph=True) 
            disc_opt.step() 

          
            gen_opt.zero_grad()
            gen_loss = get_gen_loss(gen, disc, real, condition, adv_criterion, recon_criterion, lambda_pix)
            gen_loss.backward() 
            gen_opt.step() 


            mean_discriminator_loss += disc_loss.item() / display_step
            mean_generator_loss += gen_loss.item() / display_step

            if cur_step % display_step == 0:
                if cur_step > 0:
                    print(f"Epoch {epoch}: Step {cur_step}: Generator (U-Net) loss: {mean_generator_loss}, Discriminator loss: {mean_discriminator_loss}")
                else:
                    print("Pretrained initial state")
                show_tensor_images(condition,xlabel="input")
                show_tensor_images(real,xlabel="real")
                show_tensor_images(fake,xlabel="output")
                mean_generator_loss = 0
                mean_discriminator_loss = 0
            cur_step += 1


In [ ]:
test_discriminator = patchgan(10, 1)
assert tuple(test_discriminator(
    torch.randn(1, 5, 256, 256), 
    torch.randn(1, 5, 256, 256)
).shape) == (1, 1, 16, 16)
print("Success!")

Success!


In [ ]:
x=gen(sample[0])

In [ ]:
x=x[0].detach().cpu()
x=x.reshape((256,256,3))

In [ ]:
plt.imshow(x)

In [ ]:
r=torch.ones((3,3,64,64))
x=crop(sample[0],r.shape)

In [ ]:
plt.imshow(x[0].permute(1,2,0))

In [ ]:
x.shape

In [ ]:
x55=torch.ones((3,1024,8,8))
x66=torch.ones((3,2048,4,4))

In [ ]:
up=nn.Upsample(scale_factor=2,mode='bilinear',align_corners=True)
up1=up(x66)
con=nn.Conv2d(2048,2048//2,kernel_size=2)
c=con(up1)

In [ ]:
up1.shape

In [ ]:
c.shape

In [ ]:
crp=crop(x55,c.shape)

In [ ]:
crp.shape

In [ ]:
x=torch.cat([crp,c],axis=1)

In [ ]:
x.shape

In [ ]:
con2=nn.Conv2d(2048,2048//2,kernel_size=3,padding=1)
x=con2(x)
x.shape

In [ ]:
con3=nn.Conv2d(2048//2,2048//2,kernel_size=2,padding=1)

In [ ]:
x=con3(x)
x.shape

In [ ]:
sample=next(iter(dataload))

In [ ]:

sample=next(iter(dataload))
gen=unet(3,3)

In [ ]:
condition = nn.functional.interpolate(sample[0][0], size=target_shape)
plt.imshow(condition.permute(1,2,0))